In [9]:
import pandas as pd
import csv
from IPython.display import clear_output
import os
import platform


In [10]:
file_list = os.listdir(r'.')
print(file_list)

top_letters = ['e','a','r','i','o','t','n','s','l','c','u','d','p','m','h']

['.git', '.gitignore', 'wordle helper.ipynb', 'wordle list2.csv', 'wordle_throuple.csv']


In [11]:
with open("wordle list2.csv",'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    wordle_words = list(csv_reader)[0]

In [12]:
wordle_df = pd.DataFrame(wordle_words, columns=['words'])
wordle_words = wordle_df.loc[
   (wordle_df['words'].str[0].isin(top_letters)) 
   & (wordle_df['words'].str[1].isin(top_letters)) 
   & (wordle_df['words'].str[2].isin(top_letters))
   & (wordle_df['words'].str[3].isin(top_letters))
   & (wordle_df['words'].str[4].isin(top_letters)), :
]

wordle_words = wordle_words['words'].to_list()

In [13]:
if 'wordle_throuple.csv' not in file_list:
    top_letters = ['e','a','r','i','o','t','n','s','l','c','u','d','p','m','h']


    poss_starters = []

    for ind, word in enumerate(wordle_words):
        clear_output()
        print(f'evaluating {ind} of {len(wordle_words)}: {word}')
        score = 0
        for letter in top_letters:
            if letter in word:
                score += 1
        for letter_in_word in word:
            if word.count(letter_in_word) > 1:
                score -= 1
        if score == 5:
            poss_starters.append(word)
            print('found one!')
    


working_combos = []

for i in range(len(poss_starters)-1):
    starter = poss_starters[i]
   
    j = i+1
    while j < len(poss_starters):
        compare_word = poss_starters[j]
        
        for letter in starter:
            num = compare_word.count(letter)
            if num > 0:
                break
            elif letter == starter[4] and num == 0:
                working_combos.append([starter, compare_word])
        j=j+1

print(len(working_combos))



In [14]:
if 'wordle_throuple.csv' not in file_list:
    matches = pd.DataFrame()

    for i in range(len(poss_starters)-1):
        start_word = poss_starters[i]
        match_words = poss_starters[i+1:]

        possible_matches = pd.DataFrame({
            'word':start_word,
            'matches':match_words
        })

        for letter in start_word:
            possible_matches = possible_matches[~possible_matches['matches'].str.contains(letter)]

        matches = pd.concat([matches, possible_matches], ignore_index=True).reset_index(drop=True)

    print(matches)


In [15]:
if 'wordle_throuple.csv' not in file_list:
    triple_matches = pd.DataFrame()

    for ind, row in matches.iterrows():
        poss_trip_matches = pd.DataFrame({
            'word':row['word'],
            'match_1': row['matches'],
            'match_2': matches.loc[matches['word']==row['matches'], 'matches']
        })
        for letter in row['word']:
            poss_trip_matches = poss_trip_matches[~poss_trip_matches['match_2'].str.contains(letter)]
        triple_matches = pd.concat([triple_matches, poss_trip_matches]).reset_index(drop=True)
        if ind % 100 == 0:
            clear_output()
            print(f'Running! {(ind / len(matches) * 100)}% complete!')

    triple_matches.to_csv('wordle_throuple.csv')

# if platform.system() == 'Windows':
#     os.startfile(r'wordle_throuple.csv')
# else:
#     os.system('libreoffice wordle_throuple.csv &')
    
from random import randint
triples = pd.read_csv('wordle_throuple.csv')
rand_int = randint(0, len(triples))
print(triples.loc[rand_int,:])

Unnamed: 0    30193
word          cramp
match_1       shoed
match_2       until
Name: 30193, dtype: object


combos_df = pd.DataFrame(data = working_combos, columns=['starter', 'match'])
print(combos_df)

In [16]:
not_in_word = 'cupldontrims'
not_in_word = [x for x in not_in_word]

in_word = 'eah'
in_word = [y for y in in_word]

letter_in_correct_pos = {
   
}

letters_not_in_pos = {
    3:'e',
    8:'a',
    4:'h'
}

wordle_words_df = pd.DataFrame(data=wordle_words, columns=['words'])

for l in not_in_word:
    wordle_words_df = wordle_words_df[~wordle_words_df['words'].str.contains(l, regex=True)]

temp_solutions = pd.DataFrame()

for ind, word in wordle_words_df.iterrows():
    score = 0
    for l in in_word:
        if l in word[0]:
            score = score + 1
        else:
            break
    if score == len(in_word):
        temp_solutions = pd.concat([temp_solutions, word], ignore_index = True)

for ind, l in letters_not_in_pos.items():
    temp_solutions = temp_solutions[temp_solutions[0].str[(ind % 5)] != l]

for ind, l in letter_in_correct_pos.items():
    temp_solutions = temp_solutions[temp_solutions[0].str[(ind % 5)] == l]

wordle_words_df = temp_solutions.reset_index(drop = True)

print(wordle_words_df)

KeyError: 0